In [22]:
from pathlib import Path
import pandas as pd
from dateutil.relativedelta import relativedelta
import plotly.express as px

ruta = Path(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\data\data.xlsx")
df = pd.read_excel(
    ruta, sheet_name="horizonte",
    parse_dates=["Inicio", "Fin"],
    engine="openpyxl"
).rename(columns=str.strip)

df.columns

Index(['Etapa', 'Inicio', 'Fin'], dtype='object')

In [24]:
import pandas as pd
import plotly.express as px
from dateutil.relativedelta import relativedelta

# ─── 2. DURACIÓN Y TEXTO ───────────────────────────────────
df['Duración'] = df.apply(
    lambda r: f"{relativedelta(r['Fin'], r['Inicio']).years}a "
              f"{relativedelta(r['Fin'], r['Inicio']).months}m "
              f"{relativedelta(r['Fin'], r['Inicio']).days}d", axis=1
)

df['texto_anotacion'] = "Fin: " + df['Fin'].dt.strftime("%d-%b-%Y")

# ─── 3. GRÁFICO ─────────────────────────────────────────────
fig = px.timeline(
    df.sort_values(by="Inicio"),
    x_start="Inicio",
    x_end="Fin",
    y="Etapa",
    color="Etapa",
    hover_data={"Duración": True, "Etapa": True, "Inicio": "|%d %B %Y", "Fin": "|%d %B %Y"},
    labels={"Etapa": "Etapa del Proyecto"},
)

fig.update_yaxes(showticklabels=False)

# ─── 4. POSICIÓN FINAL DE ANOTACIONES ──────────────────────
xmax = df["Fin"].max()

for i, row in df.iterrows():
    fig.add_annotation(
        x=xmax,
        y=row['Etapa'],
        text=row['texto_anotacion'],
        showarrow=False,
        font=dict(
            size=11,                # Tamaño de fuente
            color="white",
            family="Segoe UI"      # Tipo de letra
        ),
        xanchor="right",
        yanchor="middle",
        xshift=-5,
        bgcolor="rgba(50,50,50,0.7)",  # Fondo simulado (solo si usas `annotation`)
        borderpad=4
    )

# ─── 5. ESTILO ──────────────────────────────────────────────
fig.update_layout(
    title="HORIZONTE DE EVALUACIÓN DEL PROYECTO",
    title_x=0.5,
    autosize=True,
    yaxis_visible=False,
    height=None,  # ← evita height fijo
    margin=dict(t=60, r=20, l=40, b=40),
    plot_bgcolor="#0d1117",
    paper_bgcolor="#0d1117",
    font=dict(color="#c9d1d9"),
    legend_title_text='Etapas',
)
# ─── EXPORTACIÓN A HTML ───────────────────────────────────
fig.write_html(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\docs\plots\gantt_akirakato.html",
    full_html=True,
    include_plotlyjs="cdn",
    config={"responsive": True}
)
fig.show()


In [60]:
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import qualitative, sequential
from pathlib import Path

# ─── PALETAS FIJAS PARA TODA LA TESIS ───────────────────────
PALETA_CATEGORICA = qualitative.Safe      # para categorías (etapas, niveles, regiones)
PALETA_NUMERICA   = sequential.Viridis    # para mapas, proporciones, ratios

# ─── 1. CARGA ────────────────────────────────────────────────
df = pd.read_excel(
    Path(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\data\data.xlsx"),
    sheet_name="demandacp"
)

# ─── 2. PIVOTEO Y ORDEN DE GRADOS ────────────────────────────
df["grado"] = df["grado"].astype(str) + "° grado"
orden = sorted(df["grado"].unique())
df_pivot = df.pivot(index="año", columns="grado", values="total").fillna(0)
df_pivot = df_pivot[orden]

# ─── 3. CENTRADO (CAMPANA) ───────────────────────────────────
totales = df_pivot.sum(axis=1)
bottoms = -totales / 2

# ─── 4. PALETA DE COLORES COHERENTE ──────────────────────────
color_map = {g: PALETA_CATEGORICA[i % len(PALETA_CATEGORICA)] for i, g in enumerate(orden)}

# ─── 5. CREACIÓN DEL GRÁFICO ─────────────────────────────────
fig = go.Figure()

for grado in orden:
    valores = df_pivot[grado]
    fig.add_bar(
        x=df_pivot.index,
        y=valores,
        base=bottoms,
        name=str(grado),
        marker_color=color_map[grado],
        text=valores,
        textposition="inside",
        textfont=dict(color="white", size=14)
    )
    bottoms += valores

# ─── 6. LÍNEAS VERTICALES POR AÑO ────────────────────────────
for año in df_pivot.index:
    fig.add_shape(
        type="line",
        x0=año, x1=año,
        y0=-totales.max() / 2,
        y1=totales.max() / 2,
        line=dict(color="rgba(255,255,255,0.1)", width=10),
        layer="below"
    )

# ─── 7. ESTILO GENERAL ───────────────────────────────────────
fig.update_layout(
    title="DEMANDA EFECTIVA CON PROYECTO - BRECHA A ATENDER",
    title_x=0.5,
    barmode="stack",
    xaxis=dict(tickmode='linear', dtick=1),
    yaxis_visible=False,
    xaxis_visible=True,
    yaxis_showticklabels=False,
    plot_bgcolor="#0d1117",
    paper_bgcolor="#0d1117",
    font=dict(color="#c9d1d9"),
    autosize=True,
    height=None,  # ← evita height fijo
    margin=dict(t=60, r=20, l=40, b=40),
    legend_title="Grado"
)

# ─── 8. EXPORTACIÓN A HTML ───────────────────────────────────
fig.write_html(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\docs\plots\demanda_efectiva_cp.html",
    full_html=True,
    include_plotlyjs="cdn",
    config={"responsive": True}
)

fig.show()


In [56]:
import plotly.graph_objects as go
import pandas as pd

# Datos de ejemplo
df = pd.DataFrame({
    'Factores': ['Secciones', 'Aulas (B+R+M)', 'Docentes', 'Directivos + Administrativos'],
    'Antes del Proyecto': [22, 11, 35, 5],
    'Después del Proyecto': [45, 25, 62, 20]
})

# Crear figura
fig = go.Figure()

# Agregar barra "Antes del Proyecto"
# Agregar barra "Después del Proyecto"
fig.add_bar(
    x=df['Factores'],
    y=df['Después del Proyecto'],
    name='Después del Proyecto',
    marker_color="#B85555",
    text=df['Después del Proyecto'],
    textposition='inside',
    opacity=0.7,
    width=0.8
)

fig.add_bar(
    x=df['Factores'],
    y=df['Antes del Proyecto'],
    name='Antes del Proyecto',
    marker_color="#4E3ED6",
    text=df['Antes del Proyecto'],
    textposition='inside',
    width=0.5
)



# Layout
fig.update_layout(
    barmode='overlay',
    title={
        'text': 'COMPARATIVO: FACTORES PRODUCTIVOS ANTES Y DESPUÉS DEL PROYECTO',
        'x': 0.5
    },
    xaxis_title='Factores',
    yaxis_title='Cantidad',
    legend_title_text='',
    yaxis_visible=False,
    xaxis_visible=True,
    yaxis_showticklabels=False,
    plot_bgcolor="#0d1117",
    paper_bgcolor="#0d1117",
    font=dict(color="#c9d1d9"),
    autosize=True,
    height=None,  # ← evita height fijo
    margin=dict(t=60, r=20, l=40, b=40),
    legend_title="Grado"
)

# Estilo de texto
fig.update_traces(textfont_size=14)
fig.write_html(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\docs\plots\antes_despues.html", full_html=True,
               include_plotlyjs="cdn",
               config={"responsive": True})
fig.show()
